In [4]:
from typing import *
import os
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import swifter
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from tqdm.notebook import tqdm
from nptyping import NDArray, Shape, Int, Float
from rich import print as rprint

sns.set(font='IPAexGothic', style='whitegrid')
colors = ['#de3838', '#007bc3', '#ffd12a']
markers = ['o', 'x', ',']
%config InlineBackend.figure_formats = ['svg']

cmap = sns.diverging_palette(255, 0, as_cmap=True)  # カラーパレットの定義
DATA_DIR = Path()/'..'/'..'/'data'
DF_DIR = DATA_DIR/'dataframes'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

np.random.seed(1)

In [11]:
# 61

from itertools import product

from pulp import LpVariable, lpSum, value, LpProblem
from ortoolpy import model_min, addvars, addvals


df_tc: pd.DataFrame = pd.read_csv(DATA_DIR/'trans_cost.csv', index_col='工場')
df_demand: pd.DataFrame = pd.read_csv(DATA_DIR/'demand.csv')
df_supply: pd.DataFrame = pd.read_csv(DATA_DIR/'supply.csv')

display(df_tc)
display(df_demand)
display(df_supply)

nw: int = len(df_tc.index)
nf: int = len(df_tc.columns)
pr: List[Tuple[int, int]] = list(product(range(nw), range(nf)))

m1: LpProblem = model_min()
v1: Dict[Tuple[int, int], LpVariable] = {(i, j): LpVariable(f'v{i}_{j}', lowBound=0) for i, j in pr}

m1 += lpSum(df_tc.iloc[i][j] * v1[i, j] for i, j in pr)

for i in range(nw):
    m1 += lpSum(v1[i, j] for j in range(nf)) <= df_supply.iloc[0][i]
for j in range(nf):
    m1 += lpSum(v1[i, j] for i in range(nw)) >= df_demand.iloc[0][j]
m1.solve()


df_tr_solve: pd.DataFrame = df_tc.copy()
total_cost = 0
for k, x in v1.items():
    i, j = k[0], k[1]
    df_tr_solve.iloc[i][j] = value(x)
    total_cost += df_tc.iloc[i][j] * value(x)

display(df_tr_solve)
print(f'総輸送コスト: {total_cost}')

,F1,F2,F3,F4
工場,,,,
W1,10,10,11,27
W2,18,21,12,14
W3,15,12,14,12


,F1,F2,F3,F4
0,28,29,31,25


,W1,W2,W3
0,35,41,42


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jo/GitHub/knock_da/.venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/9f/gy5kzjbn1tg605r5394d47q00000gn/T/95400714f7ad4d2ca1f09a93fc0239fd-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/9f/gy5kzjbn1tg605r5394d47q00000gn/T/95400714f7ad4d2ca1f09a93fc0239fd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 49 RHS
At line 57 BOUNDS
At line 58 ENDATA
Problem MODEL has 7 rows, 12 columns and 24 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (0) rows, 12 (0) columns and 24 (0) elements
0  Obj 0 Primal inf 113 (4)
7  Obj 1296
Optimal - objective value 1296
Optimal objective 1296 - 7 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01



,F1,F2,F3,F4
工場,,,,
W1,28,7,0,0
W2,0,0,31,5
W3,0,22,0,20


総輸送コスト: 1296.0
